In [27]:
import seaborn as sns
df=sns.load_dataset('tips')

In [28]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [30]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [31]:
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [32]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [33]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [34]:
## feature engineering( Label encodeing and onehot encodeing)
## label encodeing is used when we have only two categorical group ...so in that case ..we use 0 and 1 for encoding ...but in OneHot Encoding ..when we have more that two group...like in this dataset ..day has four group..so in this case we ue this ...becaues ..by using it..one group is true make it 1 rest is 0.


In [35]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [36]:
## independet and deoendent features

In [37]:
X=df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y=df['total_bill']

In [38]:
# train_test split 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=10)

In [39]:
X_train

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2
...,...,...,...,...,...,...
64,2.64,Male,No,Sat,Dinner,3
15,3.92,Male,No,Sun,Dinner,2
228,2.72,Male,No,Sat,Dinner,2
125,4.20,Female,No,Thur,Lunch,6


In [40]:
## always do feature enginnering after train test split ..beacuese of data leakage
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
le2=LabelEncoder()
le3=LabelEncoder()


In [41]:
X_train['sex']=le1.fit_transform(X_train['sex'])
X_train['time']=le3.fit_transform(X_train['time'])
X_train['smoker']=le2.fit_transform(X_train['smoker'])

In [42]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [43]:
X_test['sex']=le1.transform(X_test['sex'])
X_test['time']=le3.transform(X_test['time'])
X_test['smoker']=le2.transform(X_test['smoker'])

In [44]:
## OneHot Encoding ---> columnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [45]:
ct = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(drop='first'), ['day'])  # or [3] if you're indexing by position
], remainder='passthrough')

In [46]:
import sys
import numpy as np 
np.set_printoptions(threshold=sys.maxsize)
X_train=ct.fit_transform(X_train)

In [47]:
X_test=ct.transform(X_test)

In [48]:
## support vector regressor 
from sklearn.svm import SVR
svr=SVR()
svr.fit(X_train,y_train)
y_pred=svr.predict(X_test)

In [49]:
from sklearn.metrics import r2_score,accuracy_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))

0.4602811456115927
4.1486423210190235
45.69213615168339


In [51]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'C':[0.1,1,10,100,1000],
    'gamma':[1,0.1,0.01,0.001,0.0001],
    'kernel':['rbf']

}

grid=GridSearchCV(SVR(),param_grid=param_grid,cv=5,verbose=3,refit=True)

grid.fit(X_train,y_train)


grid.best_params_

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.010 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.124 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.050 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.053 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

In [52]:
grid_pred=grid.predict(X_test)

In [53]:
print(r2_score(y_test,grid_pred))
print(mean_absolute_error(y_test,grid_pred))
print(mean_squared_error(y_test,grid_pred))

0.5081599655420066
3.8685177092539527
41.63875624610011
